In [1]:
#subscribe all needed topics

import rospy
from std_msgs.msg import String
from sensor_msgs.msg import Image, CameraInfo, PointCloud2
from cv_bridge import CvBridge, CvBridgeError
import message_filters
import numpy as np
from python_msg_conversions import pointclouds



def callback(l_image, camera_info, lidar):
        try:    
            bridge = CvBridge()
            cv_image = bridge.imgmsg_to_cv2(l_image, "bgr8")
            points = pointcloud2_to_xyz_array(lidar)
        except CvBridgeError as e:
            print(e)
        points = pointcloud2_to_xyz_array(lidar)
        


def listener():
        rospy.init_node('listener')
        print "loop1"
        l_image = message_filters.Subscriber("/kitti/camera_color_left/image_raw", Image )
        camera_info = message_filters.Subscriber("/kitti/camera_gray_left/camera_info", CameraInfo)
        lidar = message_filters.Subscriber("/kitti/velo/pointcloud", PointCloud2)
        ts = message_filters.ApproximateTimeSynchronizer([l_image, camera_info, lidar], 10, 0.1)
        ts.registerCallback(callback)
#         print "loop1"
        rospy.spin()

if __name__ == '__main__':
        listener()


loop1


[ERROR] [1522062639.219566]: bad callback: <bound method Subscriber.callback of <message_filters.Subscriber object at 0x7f7a918c7d10>>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/message_filters/__init__.py", line 75, in callback
    self.signalMessage(msg)
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/message_filters/__init__.py", line 57, in signalMessage
    cb(*(msg + args))
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/message_filters/__init__.py", line 287, in add
    self.signalMessage(*msgs)
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/message_filters/__init__.py", line 57, in signalMessage
    cb(*(msg + args))
  File "<ipython-input-1-9a1debe084b1>", line 57, in callback
    points = pointcloud2_to_xyz_array(lidar)
  File "<ipython-input-1-9a1debe084b1>", line 51, in pointcloud2_to_xyz_array
    retu